In [15]:
import sys
sys.path.append('..')
from dataset_reader import Traces_Dataset
from DE_obj_model import de_obj_model   
import numpy as np
import matplotlib.pyplot as plt

from exp_generate_data import exp_formalism_data_generator
from exp_hh_model import HH_model_exp

In [16]:
dataset = Traces_Dataset('../dataset_test.csv')

# sample = 1
# params = dataset.params.numpy()
# current_traces = dataset.current_traces.numpy()
# time_traces = dataset.time_traces.numpy()

# prestep_V_2d = dataset.prestep_V[sample].numpy().reshape(-1,1)
# step_Vs_2d = dataset.step_Vs[sample].numpy().reshape(-1,1)
# t = time_traces[sample]
# # target_traces = current_traces[sample]
# target_params = params[sample]

In [17]:
# # sim setup for obj evaluation model
# sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}

# # model = de_obj_model(target_params, sim_setup_2d)

In [18]:
target_num = 2
target_prestep_V = int(dataset.prestep_V[target_num])
target_step_V1 = int(dataset.step_Vs[target_num].numpy()[0])
target_dV = dataset.step_Vs[target_num].numpy()[1] - dataset.step_Vs[target_num].numpy()[0]
target_prestep_V, target_step_V1, target_dV

(-122, -41, 11.0)

In [19]:
t = np.arange(0.0, 6.0, 0.01)
target_params = dataset.params[target_num].numpy()

target_sim_setup = {'prestep_V': target_prestep_V, 'step_Vs': dataset.step_Vs[target_num].numpy(), 't':t}
params = {'p': target_params[0], 
          'g_max': target_params[1], 
          'E_rev': target_params[2], 
                 'a_m': target_params[3], 
                 'b_m': target_params[4], 
                 'delta_m': target_params[5], 
                 's_m': target_params[6]}
print(params)

model = HH_model_exp(params, target_sim_setup)
model.simulation()
if model.check_current_ss() and model.check_steady_state_curve(): 
    target_max_ind_arr = model.max_index_array
    target_current_traces = model.current_traces

target_max_ind_arr.shape, target_current_traces.shape

{'p': 4.0, 'g_max': 106.22407, 'E_rev': -71.32051, 'a_m': 2.2097604, 'b_m': 2.8441272, 'delta_m': 0.96789503, 's_m': -14.623417}


/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/Identifiability /../exp_hh_model.py:103: RuntimeWarning: divide by zero encountered in divide
  diff_arr = np.abs((self.current_traces[i, :][1:] - self.current_traces[i, :][:-1]) / (self.current_traces[i, :][:-1] - self.current_traces[i, :][0]))


((8,), (8, 600))

In [20]:
target_max_ind_arr

array([291, 284, 266, 232, 181, 125,  77,  44])

In [21]:
def obj_trace(trail_current_traces, trail_max_ind_arr, trace):
    '''
    find the area difference in the rising phase instead of the whole simulation. 
    This can be done by defining a window using the earlier threshold between two sample traces. 
    the fct is comparing two traces from the same experiment in two different samples. 
    '''
    # find the earlier threshold index + 1. 
    if trail_max_ind_arr[trace] == -1: 
        stop_ind = int(target_max_ind_arr[trace])
    elif target_max_ind_arr[trace] == -1: 
        stop_ind = int(trail_max_ind_arr[trace])
    else:
        stop_ind = int(min(trail_max_ind_arr[trace], target_max_ind_arr[trace]))
    #print(sample1, sample2, trace)
    #print(np.where(selected_t[sample1, trace] == -1)[0][0], np.where(selected_t[sample2, trace] == -1)[0][0])
    # we calculate the (avg over stop_ind points) 

    # diff_area_one_trace = np.trapz(np.abs(selected_current_traces_3d[sample1, trace, :stop_ind] - selected_current_traces_3d[sample2, trace, :stop_ind]), dataset_generator.t[:stop_ind]) / np.trapz(np.maximum(selected_current_traces_3d[sample1, trace, :stop_ind], selected_current_traces_3d[sample2, trace, :stop_ind]), dataset_generator.t[:stop_ind])
    diff_area_one_trace = np.trapz(np.abs(trail_current_traces[trace, :stop_ind] - target_current_traces[trace, :stop_ind]), t[:stop_ind]) / np.trapz(np.maximum(trail_current_traces[trace, :stop_ind], target_current_traces[trace, :stop_ind]), t[:stop_ind])
    
    
    return diff_area_one_trace

In [22]:
def obj(x, *args):
    '''
    x: trail parameters
    *args=(target_sim_setup, target_current_traces, target_max_ind_arr)
    ''' 
    params = {'p': x[0], 'g_max': x[1], 'E_rev': x[2], 'a_m': x[3], 'b_m': x[4], 'delta_m': x[5], 's_m': x[6]}
    model = HH_model_exp(params, target_sim_setup)
    trail_current_traces = model.simulation()
    trail_max_ind_arr = model.threshold_pos()
    
    diff_area = np.mean([obj_trace(trail_current_traces, trail_max_ind_arr, trace) for trace in range(target_current_traces.shape[0])])
    # diff_params_square = np.sum(np.square(selected_params[sample1] - selected_params[sample2]))
    return diff_area #, diff_params_square

In [23]:
target_params

array([  4.        , 106.22407   , -71.32051   ,   2.2097604 ,
         2.8441272 ,   0.96789503, -14.623417  ], dtype=float32)

In [24]:
x = target_params
params = {'p': x[0], 'g_max': x[1], 'E_rev': x[2], 'a_m': x[3], 'b_m': x[4], 'delta_m': x[5], 's_m': x[6]}
model = HH_model_exp(params, target_sim_setup)
trail_current_traces = model.simulation()
trail_max_ind_arr = model.threshold_pos()

/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/Identifiability /../exp_hh_model.py:84: RuntimeWarning: divide by zero encountered in divide
  diff_arr = np.abs((self.current_traces[i, :][1:] - self.current_traces[i, :][:-1]) / (self.current_traces[i, :][:-1] - self.current_traces[i, :][0]))


In [25]:
model.check_current_ss()

True

In [26]:
trail_max_ind_arr

array([291, 284, 266, 232, 181, 125,  77,  44])

In [35]:
x = dataset.params[5].numpy()
obj(x, target_sim_setup, target_current_traces, target_max_ind_arr)

0.6207174942207995

In [58]:
# these bounds are from the distribution of the params in the dataset used for NN training
params_searching_bounds = {
    'p': (1, 4),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 100), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-100, 0)
}
bounds = [params_searching_bounds['p'], params_searching_bounds['g_max'], params_searching_bounds['E_rev'], params_searching_bounds['a_m'], params_searching_bounds['b_m'], params_searching_bounds['delta_m'], params_searching_bounds['s_m']]

In [54]:
def callback_function(xk, convergence):
    callback_function.iteration += 1
    print("Iteration:", callback_function.iteration)
    print("Solution:", xk)
    print("Objective value:", obj(xk, target_sim_setup, target_current_traces, target_max_ind_arr))
    print("------------------------")
callback_function.iteration = 0  # Initialize the iteration counter

In [66]:
from scipy.optimize import differential_evolution
result = differential_evolution(obj, bounds, args=(target_sim_setup, target_current_traces, target_max_ind_arr), strategy='best1bin', maxiter=10000)#, callback=callback_function)

/var/folders/7_/yd2c8wld3sggx9gtmv9ydwm80000gn/T/ipykernel_88436/417898969.py:19: RuntimeWarning: invalid value encountered in double_scalars
  diff_area_one_trace = np.trapz(np.abs(trail_current_traces[trace, :stop_ind] - target_current_traces[trace, :stop_ind]), t[:stop_ind]) / np.trapz(np.maximum(trail_current_traces[trace, :stop_ind], target_current_traces[trace, :stop_ind]), t[:stop_ind])


In [68]:
result

 message: Maximum number of iterations has been exceeded.
 success: False
     fun: nan
       x: [ 2.190e+00  1.113e+02 -9.321e+01  6.393e+01  7.948e+01
            9.943e-01 -5.290e+00]
     nit: 10000
    nfev: 1050273

In [69]:
target_params

array([  4.        , 106.22407   , -71.32051   ,   2.2097604 ,
         2.8441272 ,   0.96789503, -14.623417  ], dtype=float32)